In [1]:
#@markdown Get the data

# run in a cron job after get_binance with github actions
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime
import pytz

# %%
#df = pd.read_csv("BTCUSDT_historical.csv", index_col="Unnamed: 0", parse_dates=True)
# The data is in UTC time
df = pd.read_csv("https://raw.githubusercontent.com/sotoblanco/Binance_data_live/main/data/BTCUSDT_historical_1h.csv",
                 index_col="Unnamed: 0", parse_dates=True)
# convert the data into EST time
df.index = df.index - pd.to_timedelta(4, unit='h')

df_day = df[(df.index.time >= datetime.time(9))
   & (df.index.time <= datetime.time(16))]

df_night = df[(df.index.time >= datetime.time(17)) |
            (df.index.time <= datetime.time(8))]


# %%
df_day["date"] = pd.to_datetime(df_day.index.date)

# %%
df_night["date"] = pd.to_datetime(df_night.index.date)

df_night["date"] = df_night.index - pd.to_timedelta(9, unit="h")

# %%
df_night.index = df_night.date

# %%
ohlc_dict = {
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum"}

day_sum = df_day.resample('1d').apply(ohlc_dict)
night_sum = df_night.resample('1d').apply(ohlc_dict)

# %%
#night_sum = night_sum.shift().dropna()

# %%
day_night = day_sum.merge(night_sum, left_index=True, right_index=True, suffixes=('_day', '_night'))

current_time = (datetime.datetime.now(pytz.timezone('US/Pacific')) - pd.to_timedelta(4, 'h')).strftime("%Y-%m-%d")

### Get the distance between the previous day and the current day

features = ["high_day", "low_day", "high_night", "low_night"]

previous_day = (datetime.datetime.now(pytz.timezone('US/Pacific')) - pd.to_timedelta(1, 'd')).strftime("%Y-%m-%d")
previous_day_df = day_night[day_night.index == previous_day]

current_day = df_day[df_day.date == current_time]
current_open = current_day.iloc[0]['open']

pred_dict = {}
for col in features:
    pred_dict[f'ret_distance_p{col}_open'] = (current_open - previous_day_df[col].values[0])/previous_day_df[col].values[0]

In [3]:
#@markdown Create the prediction and export the chart
import requests
import json

url = 'http://3.232.96.214:3000/classify'
#url = 'http://localhost:3000/classify'

#data = dict(last_day)

j_data = json.dumps(pred_dict)
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=j_data, headers=headers)

current_day = df_day[df_day.index  >= day_sum.index[-1]]

previous_day = day_night[day_night.index < current_day['date'].iloc[-1]]

predicted_value =  [previous_day.iloc[-1]['high_day']] * len(current_day)

# Show graph of the day
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#warnings.filterwarnings("ignore")
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(xaxis_rangeslider_visible=False, yaxis_tickformat = '.')

fig.add_trace(go.Candlestick(x=current_day.index,
                open=current_day['open'],
                high=current_day['high'],
                low=current_day['low'],
                close=current_day['close']))
                
fig.add_trace(go.Line(x=current_day.index, y=predicted_value, name = "pHOD",
                      line=dict(color='darkslateblue', width=4, dash='dot')),
                      secondary_y=False)

fig.add_annotation(x=current_day.index[-1], y=predicted_value[-1], text=f"{round(float(r.text),4)}", 
                        font=dict(family="Courier New, monospace",
                                    color="#ffffff"), bordercolor="#c7c7c7",
                                    borderwidth=2,
                                borderpad=4,
                                bgcolor="#ff7f0e",
                                opacity=0.8)

c:\Users\Pastor\anaconda3\envs\ml-zoomcamp\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


